In [25]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [26]:
iOption0 = 0
is_sigmalised = 0
sim = 0
new = 1

In [27]:
isigma = 2
N_files = 1
N_centr = 8
N_hists = 4
N_layers = 6
N_ilyaers = 1
is_e = 1
centralities = ["1-2 layer pos","1-2 layer neg","2-3 layer pos","2-3 layer neg","2-4 layer pos","2-4 layer neg","3-4 layer pos","3-4 layer neg","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5,6,6]
layer_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
layer_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if is_sigmalised: layer_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/rg/"
file_names="my-27th_QA_19200_350runs" #my-25th_QA_19197_1000runs
if new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new5_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [28]:
if sim :
    file_path="../sim/"
    file_names="my-kek"

In [29]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if not new: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

input/NewHadron/missalign_hadron_new5_rg0-80.root


In [30]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [31]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]

In [32]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    central_bean_prev = (0.001*layer_bins[2*ilayer]-0.0005-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{icentr}_{ilayer}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(icentr%2)][0]+central_bean_prev, boards_fit[int(icentr%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(icentr%2)][0], boards_tot[int(icentr%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus2.SetParameter(1,0.5*(boards_fit2[icentr][0]+boards_fit2[icentr][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[icentr][1]-boards_fit2[icentr][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(icentr%2)][0]/10+central_bean_prev,boards_tot[int(icentr%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(icentr%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if ilayer>-1:
            total.SetParLimits(1,boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if iOption0<4 and is_sigmalised==0:
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ (0.15-0.005*p)*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+(0.15-0.005*p)*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [33]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+is_sigmalised*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [34]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

In [35]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [36]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_{icentr}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if icentr<2 : fit_functions1[-1].SetParLimits(2,-3.5,-2.5)
            elif icentr<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.5)
                fit_functions1[-1].SetParLimits(1,0.001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [37]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            if iOption0 == 0 and icentr%2==0 and (ilayer == 0 or ilayer == N_layers-1): 
                g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","",0.2,1.2)
            else: g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

array('d', [0.0033274054550109238, 0.0025991753505536135, 0.002117786718542155, 0.0018838850313808472, 0.0017639941008405823, 0.0016663550622126938, 0.0016128135364388676, 0.0015604822552280492, 0.0014806885952085606, 0.0017322717080558202, 0.0042007216509521265, 0.012]) array('d', [0.004189235350808733, 0.0031450342890121464, 0.0026377225097186058, 0.0022567806170145375, 0.0019772624289718763, 0.0018076963655642355, 0.001680008161181567, 0.0016005967598393734, 0.0014721450441557136, 0.0012765164627004013, 0.0012000000074099163, 0.0012000064024417305])


NameError: name 'N_file' is not defined

In [ ]:
print(f"sigma_pt_params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][8][6][3]={
{
{
{0.001402, 0.003827, -3.5},
{0.001324, 0.003794, -3.239219},
{0.001252, 0.003495, -2.848446},
{0.001257, 0.00355, -2.890077},
{0.001268, 0.003402, -2.822655},
{0.001417, 0.003781, -3.5},
},
{
{0.001261, 0.003568, -2.952267},
{0.001242, 0.003544, -2.89512},
{0.001267, 0.00363, -3.037877},
{0.001257, 0.003562, -2.937289},
{0.001302, 0.00363, -3.110354},
{0.001433, 0.003577, -3.5},
},
{
{0.001415, 0.006691, -3.562122},
{0.001287, 0.006493, -3.353565},
{0.001206, 0.006117, -3.116073},
{0.001193, 0.006078, -3.073923},
{0.001218, 0.006082, -3.114267},
{0.001335, 0.006058, -3.296664},
},
{
{0.001213, 0.005256, -2.845637},
{0.001197, 0.00571, -3.056406},
{0.001191, 0.005845, -3.116371},
{0.001194, 0.005906, -3.147174},
{0.001238, 0.00603, -3.2745},
{0.00141, 0.007109, -3.999999},
},
{
{0.001897, 0.009862, -3.229163},
{0.001539, 0.009792, -3.039095},
{0.001329, 0.01, -3.0},
{0.001278, 0.01, -3.0},
{0.001361, 0.01, -3.0},
{0.001724, 0.009533, -3.22576},
},
{
{0.

In [ ]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0==0:
                    fit_functions1[-1].SetParLimits(2,-5.126,-0.23)
                    if icentr%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-2.0,-1)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,0.1,1.3)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,1,2.0)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-1.3,-0.1)
                    if ilayer==3 and icentr ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        fit_functions1[-1].SetParameter(1,1.6)
                        fit_functions1[-1].SetParameter(2,-0.9)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [ ]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_mean = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_mean[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_mean[ifile][icentr][ilayer][-1].Draw('P')
            if not is_sigmalised: g_mean[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","",0.7,1)
            else: g_mean[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","",0.3,5)
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [ ]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_centr}][{N_layers}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_centr}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iicentr = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iicentr%2)].GetParameter(0),6)}, {round(fits[3-int(iicentr%2)].GetParameter(1),6)}, {round(fits[3-int(iicentr%2)].GetParameter(2),6)}"+"},")
            iicentr+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_icentr in fit_functions_ifile:
            print("{")
            for fits in fit_functions_icentr:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

mean_pt_params[1][8][6]={
{
{-0.009834, -0.004747,-0.000672, 0.00034, 0.004413, 0.009497},
{-0.009457, -0.004384,-0.000332, 0.000679, 0.004729, 0.00978},
{-0.010464, -0.005308,-0.001184, -0.00015, 0.00398, 0.009126},
{-0.009474, -0.004297,-0.000151, 0.000885, 0.005027, 0.010194},
{-0.01178, -0.006238,-0.001713, -0.000577, 0.003945, 0.009534},
{-0.010154, -0.004489,1.9e-05, 0.001141, 0.005578, 0.011054},
{-0.010936, -0.005471,-0.001056, 5.4e-05, 0.004466, 0.00994},
{-0.01026, -0.004722,-0.000328, 0.000766, 0.0051, 0.010492},
},
};


In [ ]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for ilayer in range(N_layers):
    if not is_sigmalised: dphi.append( round(0.001*layer_bins[2*ilayer]-0.0005-0.1,5))
    else: dphi.append(round(0.2*layer_bins[2*ilayer]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParameter(0))
            dynamic_dphi_err_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-0.0095, -0.0045, -0.0005, 0.0005, 0.0045, 0.0095])
[[array('d', [-0.009834265504681135, -0.004747309228042392, -0.0006721315940118947, 0.0003399597060385201, 0.004412657457362194, 0.009496596187319156]), array('d', [-0.009457331493856875, -0.004384211994546309, -0.00033195567177511306, 0.0006785140458940644, 0.00472911690560579, 0.009779906835036715]), array('d', [-0.01046379711615077, -0.005307823966016308, -0.0011836772216196959, -0.0001501853304603784, 0.0039804346028799245, 0.009126199062547639]), array('d', [-0.009474339301038668, -0.004297369022674472, -0.00015095055227537166, 0.0008847377070215386, 0.005027287959768187, 0.010194232336497615]), array('d', [-0.01178015799070314, -0.006237806003748864, -0.0017133875215508305, -0.0005770237503943294, 0.0039445208352086696, 0.009534227897397253]), array('d', [-0.010154040156282695, -0.004488625946028269, 1.9373704166061878e-05, 0.0011409131247812204, 0.005577707925407585, 0.011054250194601531]), array('d', [-0.01

In [ ]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_mean_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, dynamic_dphi[ifile][icentr],dphi_err,dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_mean_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_mean_phi[ifile][icentr][ilayer][-1].Fit("pol1","Q","")
            g_mean_phi[ifile][icentr][ilayer][-1].GetFunction("pol1").SetLineColor(colors[icentr])
            g_mean_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [ ]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_functions[ifile][icentr][ilayer].Eval(pt_point)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
            dynamic_dphi_err_icentr.append((fit_functions[ifile][icentr][ilayer].GetParError(0)*2)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [0.9875791254742001, 0.9914012152832665, 0.9998760550883043, 1.0001239449116957, 1.00217851513437, 0.9905665373905567]), array('d', [1.003204936478283, 1.0028123293131748, 0.9962866823714699, 1.0037133176285302, 0.9995034838814297, 0.9925584561729182]), array('d', [1.0180078443345502, 1.0015749298789032, 0.9986730169434035, 1.0013269830565965, 1.0010694776261366, 1.0013044187365925]), array('d', [1.0247829537671849, 1.006426188498425, 1.0005540952313405, 0.9994459047686595, 0.9964380649275536, 0.9800946401401498]), array('d', [1.0916986884332835, 1.0415149089261102, 1.0071475601592021, 0.992852439840798, 1.016325753422443, 1.0252447197111114]), array('d', [1.0352337991263851, 1.0021742740892514, 0.9942522346949451, 1.005747765305055, 1.0042633795028546, 0.9997788364606711]), array('d', [1.1738244468566903, 1.0349596471468423, 1.0020821797684092, 0.9979178202315908, 0.9998003022323421, 1.0302930137450248]), array('d', [1.0641835387342597, 1.0085088191768878, 0.9997220169131

In [ ]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_sigma_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, sigma_dynamic_dphi[ifile][icentr],dphi_err,sigma_dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_sigma_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_sigma_phi[ifile][icentr][ilayer][-1].Fit("pol2","Q","",)
            g_sigma_phi[ifile][icentr][ilayer][-1].GetFunction("pol2").SetLineColor(colors[icentr])
            g_sigma_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [ ]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_centr}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fits in fit_functions_ifile:
        print("{"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("},")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][3]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(0),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(1),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("},")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][2]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(0),6)},{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("},")
print("};")



const float phi_sigma_pt_params[1][8][3]={
{
{0.001254, 0.003523, -2.869262},
{0.001262, 0.003596, -2.987583},
{0.0012, 0.006098, -3.094998},
{0.001193, 0.005875, -3.131773},
{0.001303, 0.01, -3.0},
{0.001281, 0.01, -3.0},
{0.001181, 0.008108, -4.999998},
{0.001183, 0.008016, -4.99998},
},
};
const float phi_sigma_phi_params[1][8][3]={
{
{0.99956,0.402637,-118.65369},
{1.001198,-0.500217,-36.315815},
{0.999953,-0.294937,97.157566},
{0.99969,-2.130841,30.522771},
{1.002122,-2.710835,951.898979},
{1.006307,4.433898,-542.649564},
{0.999289,-5.84735,1043.427672},
{1.000707,0.534252,710.479926},
},
};
const float phi_mean_phi_params[1][8][2]={
{
{-0.000167,1.017468},
{0.000169,1.012502},
{-0.000667,1.031247},
{0.000364,1.035389},
{-0.001138,1.12368},
{0.000525,1.116725},
{-0.0005,1.099738},
{0.000175,1.092076},
},
};
